# How to Build Custom Context Managers For Data Science Workflow
## `with` open awesome skill
<img src='images/bike.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@gijs-coolen-1325695?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Gijs Coolen</a>
        on 
        <a href='https://www.pexels.com/photo/black-bmw-motorcycle-2948297/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>